In [ ]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import lxml
import numbers
from collections import defaultdict
from uszipcode import *
from redfin import Redfin

import sys
sys.path.insert(2, './apartments-scraper-master/')

from parse_apartments import *

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore")

from redfin import Redfin

request_headers = {
        'authority': 'www.redfin.com',
        'content-length': '0',
        'sec-ch-ua': '"Chromium";v="92", " Not A;Brand";v="99", "Google Chrome";v="92"',
        'sec-ch-ua-mobile': '?0',
        'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 13982.82.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.157 Safari/537.36',
        'content-type': 'text/plain;charset=UTF-8',
        'accept': '*/*',
        'origin': 'https://www.redfin.com',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'no-cors',
        'sec-fetch-dest': 'empty',
        'referer': 'https://www.redfin.com/city/12839/DC/Washington-DC',
        'accept-language': 'en-US,en;q=0.9',
    }

def redfin_base_info(zipcode):
    # zipcode = '98034'
    url = 'https://www.redfin.com/zipcode/{}'.format(str(zipcode))
    response = requests.get(url, headers=request_headers).text
    soup = BeautifulSoup(response, 'html.parser')
    num_page = [x.text.strip() for x in soup.findAll('span',{'class':'pageText'})]
    num_page = [i.split(' ')[-1] for i in num_page] 
    print('zipcode:{}'.format(zipcode))
    print('Num of pages:{}'.format(num_page))
    time.sleep(3)
    base_list = []
        
    for i in range(1,int(num_page[0])+1):
        page_url = url + '/page-' + str(i)
        print(page_url)
        response = requests.get(page_url, headers=request_headers).text
        soup = BeautifulSoup(response, 'html.parser')
        time.sleep(3)
        home_type = [x.text.strip() for x in soup.findAll('script',{'type':'application/ld+json'})]
    
        for i in range(len(home_type)):
            data = pd.json_normalize(json.loads(str(home_type[i]))) 
            base_list.append(data)

    final_base_list = pd.concat(base_list)
    final_base_list = final_base_list[final_base_list['address.streetAddress'].notnull()]
    final_base_list = final_base_list[['@type','name','url','numberOfRooms','address.@type','address.streetAddress','address.addressRegion','address.postalCode']]
    final_base_list.columns = ['property_type','address','url','number_rooms','address_type','street_address','address_region','postal_code']
    final_base_list['propertyId'] = final_base_list['url'].str.split('/').str[-1].astype('int')
    return final_base_list
    
def redfin_data_processing(address):
    client = Redfin()
    print(address)
    response = client.search(address)
    time.sleep(3)
    property_df = pd.DataFrame()
    condition = response['payload']['hasFakeResults']
    print('hasFakeResults:{}'.format(condition))
    try:
        url = response['payload']['exactMatch']['url']
        initial_info = client.initial_info(url)
        property_id = initial_info['payload']['propertyId']
        listing_id = initial_info['payload']['listingId']

        mls_data = client.below_the_fold(property_id)
        avm_details = client.avm_details(property_id, listing_id)
        cost_of_home_ownership = client.cost_of_home_ownership(property_id)
        info_panel = client.info_panel(property_id, listing_id)
        rental_estimate = client.rental_estimate(property_id, listing_id)

        # region_id = rental_estimate['payload']['rentalEstimateInfo']['regionId']

        mls_data_df = pd.json_normalize(mls_data)
        avm_details_df = pd.json_normalize(avm_details)
        rental_estimate_df = pd.json_normalize(rental_estimate)
        cost_of_home_ownership_df = pd.json_normalize(cost_of_home_ownership)
        info_panel_df = pd.json_normalize(info_panel)

        avm_details_df['lastSoldDate'] = pd.to_datetime(avm_details_df.get('payload.lastSoldDate', pd.Series(index=avm_details_df.index, name='lastSoldDate'))/1000, unit='s')
        avm_details_df['listingPrice'] = avm_details_df.get('payload.listingPrice', pd.Series(index=avm_details_df.index, name='listingPrice'))
        avm_details_df['lastSoldPrice'] = avm_details_df.get('payload.lastSoldPrice', pd.Series(index=avm_details_df.index, name='lastSoldPrice'))
        avm_details_df['predictedValue'] = avm_details_df.get('payload.predictedValue', pd.Series(index=avm_details_df.index, name='predictedValue'))

        property_info = avm_details_df[['payload.propertyId','payload.streetAddress.assembledAddress','payload.numBeds','payload.numBaths','payload.sqFt.value'
        ,'listingPrice','lastSoldPrice','lastSoldDate','payload.latLong.latitude','payload.latLong.longitude','payload.streetAddress.unitType'
        ,'payload.streetAddress.unitValue','predictedValue']]
        property_info_column = ['propertyId','assembledAddress','numBeds','numBaths','sqFt','listingPrice','lastSoldPrice','lastSoldDate','latitude','longitude','streetAddress_unitType','streetAddress_unitValue','predictedValue']
        property_info.columns = property_info_column

        rental_estimate_df['payload.rentalEstimateInfo.propertyId'] = rental_estimate_df.get('payload.rentalEstimateInfo.propertyId', pd.Series(index=rental_estimate_df.index, name='payload.rentalEstimateInfo.propertyId'))
        rental_estimate_df['payload.rentalEstimateInfo.rentAggregateChangeSince.year'] = rental_estimate_df.get('payload.rentalEstimateInfo.rentAggregateChangeSince.year', pd.Series(index=rental_estimate_df.index, name='payload.rentalEstimateInfo.rentAggregateChangeSince.year'))
        rental_estimate_df['payload.rentalEstimateInfo.rentAggregateChangeSince.month'] = rental_estimate_df.get('payload.rentalEstimateInfo.rentAggregateChangeSince.month', pd.Series(index=rental_estimate_df.index, name='payload.rentalEstimateInfo.rentAggregateChangeSince.month'))
        rental_estimate_df['payload.rentalEstimateInfo.rentAggregateChangeSince.day'] = rental_estimate_df.get('payload.rentalEstimateInfo.rentAggregateChangeSince.day', pd.Series(index=rental_estimate_df.index, name='payload.rentalEstimateInfo.rentAggregateChangeSince.day'))
        rental_estimate_df['payload.rentalEstimateInfo.predictedValueLow'] = rental_estimate_df.get('payload.rentalEstimateInfo.predictedValueLow', pd.Series(index=rental_estimate_df.index, name='payload.rentalEstimateInfo.predictedValueLow'))
        rental_estimate_df['payload.rentalEstimateInfo.predictedValueHigh'] = rental_estimate_df.get('payload.rentalEstimateInfo.predictedValueHigh', pd.Series(index=rental_estimate_df.index, name='payload.rentalEstimateInfo.predictedValueHigh'))
        rental_estimate_df['payload.rentalEstimateInfo.regionName'] = rental_estimate_df.get('payload.rentalEstimateInfo.regionName', pd.Series(index=rental_estimate_df.index, name='payload.rentalEstimateInfo.regionName'))
        rental_estimate_df['payload.rentalEstimateInfo.regionMedianRent'] = rental_estimate_df.get('payload.rentalEstimateInfo.regionMedianRent', pd.Series(index=rental_estimate_df.index, name='payload.rentalEstimateInfo.regionMedianRent'))
        rental_estimate_df['payload.rentalEstimateInfo.regionRentChangeAsPercentage'] = rental_estimate_df.get('payload.rentalEstimateInfo.regionRentChangeAsPercentage', pd.Series(index=rental_estimate_df.index, name='payload.rentalEstimateInfo.regionRentChangeAsPercentage'))

        rental_estimate_df['rentAggregateChangeSince'] = rental_estimate_df['payload.rentalEstimateInfo.rentAggregateChangeSince.year'].astype(str) + '-' + rental_estimate_df['payload.rentalEstimateInfo.rentAggregateChangeSince.month'].astype(str)+ '-' + rental_estimate_df['payload.rentalEstimateInfo.rentAggregateChangeSince.day'].astype(str)
        rental_info = rental_estimate_df[['payload.rentalEstimateInfo.propertyId','payload.rentalEstimateInfo.predictedValueLow'
        ,'payload.rentalEstimateInfo.predictedValueHigh','payload.rentalEstimateInfo.regionName'
        ,'payload.rentalEstimateInfo.regionMedianRent','payload.rentalEstimateInfo.regionRentChangeAsPercentage'
        ,'rentAggregateChangeSince','payload.previewText']]
        rental_info_column = ['propertyId','predictedValueLow','predictedValueHigh','regionName','regionMedianRent','regionRentChangeAsPercentage','rentAggregateChangeSince','previewText']
        rental_info.columns = rental_info_column

        info_panel_df = info_panel_df[['payload.mainHouseInfo.listingId','payload.mainHouseInfo.mlsId','payload.mainHouseInfo.propertyAddress.city'
        ,'payload.mainHouseInfo.propertyAddress.stateOrProvinceCode','payload.mainHouseInfo.propertyAddress.postalCode']]

        info_panel_list = info_panel['payload']['mainHouseInfo']['selectedAmenities']
        info_panel_list_df = []
        for i in range(0,len(info_panel_list)):
            info_panel_list_df.append(pd.json_normalize(info_panel_list[i]))

        info_panel_list_df = pd.concat(info_panel_list_df)
        info_panel_df['hoa'] = info_panel_list_df[info_panel_list_df['header']=='HOA Dues']['content']
        info_panel_df['property_type'] = info_panel_list_df[info_panel_list_df['header']=='Property Type']['content']
        info_panel_df['community'] = info_panel_list_df[info_panel_list_df['header']=='Community']['content']
        info_panel_df['year'] = info_panel_list_df[info_panel_list_df['header']=='Built']['content']
        info_panel_df['lot_size'] = info_panel_list_df[info_panel_list_df['header']=='Lot Size']['content']

        info_panel_column = ['listing_id','mlsId','city','state','postalCode','hoa','property_type','community','year','lot_size']
        info_panel_df.columns = info_panel_column
        info_panel_df['propertyId'] = property_id
        info_panel_df['hoa'] = info_panel_df['hoa'].str.extract('(\d+)')

        cost_of_home_ownership_df = cost_of_home_ownership_df[['payload.mortgageCalculatorInfo.downPaymentPercentage','payload.mortgageCalculatorInfo.propertyTaxRate'
        ,'payload.mortgageCalculatorInfo.homeInsuranceRate','payload.mortgageCalculatorInfo.mortgageInsuranceRate','payload.mortgageCalculatorInfo.creditScore'
        ,'payload.mortgageCalculatorInfo.mortgageRateInfo.fifteenYearFixed','payload.mortgageCalculatorInfo.mortgageRateInfo.fiveOneArm'
        ,'payload.mortgageCalculatorInfo.mortgageRateInfo.thirtyYearFixed']]

        cost_of_home_ownership_df_column = ['downPaymentPercentage','propertyTaxRate','homeInsuranceRate','mortgageInsuranceRate','creditScore','fifteenYearFixed','fiveOneArm','thirtyYearFixed']
        cost_of_home_ownership_df.columns = cost_of_home_ownership_df_column
        cost_of_home_ownership_df['propertyId'] = property_id

        property_df = property_info.merge(rental_info, how='left', on = 'propertyId').merge(info_panel_df, how='left', on = 'propertyId').merge(cost_of_home_ownership_df, how='left', on = 'propertyId')

        property_df['down'] = property_df['listingPrice']*(property_df['downPaymentPercentage']/100)
        property_df['loan'] = property_df['listingPrice']*(1-property_df['downPaymentPercentage']/100)
        property_df['mortgage_payment_30'] = (property_df['loan'] * (property_df['thirtyYearFixed']/1200) * ((1 + property_df['thirtyYearFixed']/1200) ** (30*12))) / ((1 + property_df['thirtyYearFixed']/1200) ** (30*12) - 1)
        property_df['property_tax'] = property_df['listingPrice'] * property_df['propertyTaxRate']/1200
        property_df['home_insurance'] = property_df['listingPrice'] * property_df['homeInsuranceRate']/1200

        property_df['hoa'].fillna(0, inplace=True)
        property_df['monthly_cost'] = property_df['mortgage_payment_30'] + property_df['property_tax'] + property_df['home_insurance'] + property_df['hoa'].astype('int')
        print('Listing price is:{}'.format(property_df['listingPrice'][0]))
    
    except:
        pass
    
    return property_df

def redfin_wrapper(zipcode):
    df = redfin_base_info(zipcode)
    output_df = []
    for i in df['address'].to_list():                           
        output_df.append(redfin_data_processing(i))

    output_df_final = pd.concat(output_df)
    output_df_final = df.merge(output_df_final, how='left',on='propertyId')
    return output_df_final